<a href="https://colab.research.google.com/github/Emmanuel-Ayeni/foulMvSoccerNet-/blob/main/ai_var_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install soccernet
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.1/922.1 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [3]:
!pwd

/content


In [4]:
!ls -lt

total 8
drwx------ 6 root root 4096 Feb 28 19:28 drive
drwxr-xr-x 1 root root 4096 Feb 27 14:22 sample_data


In [8]:
!ls -lt

total 8
drwx------ 6 root root 4096 Feb 28 19:28 drive
drwxr-xr-x 1 root root 4096 Feb 27 14:22 sample_data


In [9]:
# Libaries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import os
import zipfile
import json

import cv2  # OpenCV for video processing
import SoccerNet
import torch
from ultralytics import YOLO


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Example model
from sklearn.metrics import classification_report, accuracy_score

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [10]:
from SoccerNet.Downloader import SoccerNetDownloader as SNdl
mySNdl = SNdl(LocalDirectory="path/to/SoccerNet")
mySNdl.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password="s0cc3rn3t")

In [14]:
!ls -lt path/to/SoccerNet/mvfouls/


total 3246236
-rw-r--r-- 1 root root  245990818 Feb 28 21:48 challenge.zip
-rw-r--r-- 1 root root  268201537 Feb 28 21:35 test.zip
-rw-r--r-- 1 root root  350517090 Feb 28 21:34 valid.zip
-rw-r--r-- 1 root root 2459423137 Feb 28 21:34 train.zip


In [15]:
def extract_zip_files_to_subdirs(zip_dir, base_extract_dir):
    """
    Extracts each zip file in zip_dir to a subdirectory
    named after the zip file (without the .zip extension)
    within base_extract_dir.  No files are extracted directly
    to base_extract_dir.
    """

    try:
        for filename in os.listdir(zip_dir):
            if filename.endswith('.zip'):
                zip_filepath = os.path.join(zip_dir, filename)
                subdir_name = os.path.splitext(filename)[0]  # Name from filename
                extract_to = os.path.join(base_extract_dir, subdir_name)

                # Create the base directory if it doesn't exist.
                if not os.path.exists(base_extract_dir):
                    os.makedirs(base_extract_dir)

                if not os.path.exists(extract_to):
                    os.makedirs(extract_to)  # Create subdir if it doesn't exist.

                print(f"Extracting {filename} to {extract_to}...")

                with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)

                print(f"Extracted {filename} to {extract_to}")

    except Exception as e:
        print(f"Error during extraction: {e}")




In [16]:
zip_path = "./path/to/SoccerNet/mvfouls/"  # Replace with actual path
base_extract_dir = "./soccer_extract_data"  # Base directory for subfolders
extract_zip_files_to_subdirs(zip_path, base_extract_dir)

Extracting valid.zip to ./soccer_extract_data/valid...
Extracted valid.zip to ./soccer_extract_data/valid
Extracting test.zip to ./soccer_extract_data/test...
Extracted test.zip to ./soccer_extract_data/test
Extracting challenge.zip to ./soccer_extract_data/challenge...
Extracted challenge.zip to ./soccer_extract_data/challenge
Extracting train.zip to ./soccer_extract_data/train...
Extracted train.zip to ./soccer_extract_data/train


In [18]:
!ls -lt ./soccer_extract_data

total 120
drwxr-xr-x 2918 root root 77824 Feb 28 22:27 train
drwxr-xr-x  275 root root 12288 Feb 28 22:26 challenge
drwxr-xr-x  303 root root 12288 Feb 28 22:26 test
drwxr-xr-x  413 root root 16384 Feb 28 22:26 valid


In [19]:
!ls -lt ./soccer_extract_data/test/ # Unix command to verify the action directories and respective annotation json file

total 1552
-rw-r--r-- 1 root root 353861 Feb 28 22:26 annotations.json
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root   4096

In [20]:
!ls -lt ./soccer_extract_data/train/ # Unix command to verify the action directories and respective annotation json file

total 14948
-rw-r--r-- 1 root root 3360392 Feb 28 22:27 annotations.json
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_999
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_998
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_997
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_996
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_995
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_994
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_993
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_992
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_991
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_990
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_99
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_989
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_988
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_987
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_986
drwxr-xr-x 2 root root    4096 Feb 28 22:27 action_985
drwxr-xr-x 2 root root    4096 Feb 28 22:27 acti

In [21]:
!ls -lt ./soccer_extract_data/valid/ # Unix command to verify the action directories and respective annotation json file

total 2120
-rw-r--r-- 1 root root 483560 Feb 28 22:26 annotations.json
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root   4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root   4096

In [22]:
!ls -lt ./soccer_extract_data/challenge/ # Unix command to verify the action directories and except the annotation json file

total 1092
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_99
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_98
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_97
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_96
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_95
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_94
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_93
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_92
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_91
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_90
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_9
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_89
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_88
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_87
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_86
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_85
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_84
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_83
drwxr-xr-x 2 root root 4096 Feb 28 22:26 action_82
drwxr-xr-x 2 root roo

In [24]:
# Path to Video and JSON Annotation
video_path = "./soccer_extract_data/test/action_1.mp4"  # Replace with actual path
json_path = "./soccer_extract_data/test/annotations.json"  # Replace with actual path

In [25]:
# Load JSON Annotations
with open(json_path, 'r') as f:
    annotations = json.load(f)

In [28]:
# Video Processing (Frame Extraction and Saving)
def process_video(video_path, output_dir, frame_rate=5):  # Adjust frame rate as needed
    """Extracts frames from a video and saves them as images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_subdir = os.path.join(output_dir, video_name)
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = cap.read()

    while success:
        if frame_count % frame_rate == 0: # Extract every 'frame_rate'-th frame
            frame_filename = f"frame_{frame_count:05d}.jpg"  # Format frame number
            frame_path = os.path.join(output_subdir, frame_filename)
            cv2.imwrite(frame_path, frame)  # Save the frame

        frame_count += 1
        success, frame = cap.read() # Read the next frame
    cap.release()
    print(f"Extracted frames from {video_path} to {output_subdir}")
    return output_subdir



In [29]:
output_frames_dir = "./frames" # Directory to save extracted frames
frames_directory = process_video(video_path, output_frames_dir)

Extracted frames from ./soccer_extract_data/test/action_1.mp4 to ./frames/action_1


In [32]:
!ls -lt ./frames/

total 4
drwxr-xr-x 2 root root 4096 Feb 28 22:52 action_1


In [41]:
!ls -lt ./frames/action_1/

total 0


Data Proprocessing

In [40]:
import tensorflow
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load a pre-trained ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(video_frames):
    """
    Extract features from a sequence of video frames using ResNet50.
    """
    features = []
    for frame in video_frames:
        frame = cv2.resize(frame, (224, 224))  # Resize to ResNet input size
        frame = preprocess_input(frame)  # Preprocess for ResNet
        frame_features = base_model.predict(np.expand_dims(frame, axis=0))
        features.append(frame_features)
    return np.array(features)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
frames_directory